## code for generating ATC codes from rxcui and NDC


In [9]:
import requests
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import psycopg2
import collections
import asyncio
import getpass
import re
from datetime import datetime as dt
import os,sys,re
import urllib3
import prettytable
from collections import Counter
import seaborn as sns
import random

from sklearn.externals.joblib import Memory
memory = Memory(cachedir='/tmp', verbose=0)
#@memory.cache above any def fxn.

%matplotlib inline
plt.style.use('ggplot')

from notebook.services.config import ConfigManager
cm = ConfigManager()
cm.update('livereveal', {
        'width': 1024,
        'height': 768,
        'scroll': True,
})

%load_ext autotime

In [2]:
import pandas as pd 

## Fetch all responses within one Client session first

In [13]:
#I need help understanding what this is doing, can you elaborate?
#what are the fetch and run functions? This is used in pinging a api, right?

time: 626 µs


In [5]:
import asyncio
from aiohttp import ClientSession
import time
import json

async def fetch(url, session):
    async with session.get(url) as response:
        return await response.read()

async def run(url_lst):
    tasks = []

    # Fetch all responses within one Client session,
    # keep connection alive for all requests.
    start = time.perf_counter()
    
    async with ClientSession() as session:
        for url in url_lst:
            task = asyncio.ensure_future(fetch(url, session))
            tasks.append(task)

        responses = await asyncio.gather(*tasks)
        # you now have all response bodies in this variable
#         print(responses)

    duration = time.perf_counter() - start
    print('total time: {}'.format(duration))
    return responses

## Access MIMIC database and convert it to dataframe in Pandas

In [37]:
import psycopg2
# conn = psycopg2.connect(dbname="mimic", user="student", password="lipup", host="luolabwkstn01.fsm.northwestern.edu", port="5432")
# cur=conn.cursor()

conn = psycopg2.connect(dbname="mimic", user="student", host="luolabwkstn01.fsm.northwestern.edu", port="5432",
                        password=getpass.getpass(prompt='Password:'.format()))
cur=conn.cursor()

query_schema = 'SET search_path to ' + "mimiciii" + ';'

Password:········
time: 2.01 s


## Prescription

In [ ]:
#code for generating ATC codes from rxcui and NDC

In [38]:
cur.execute('Select * from mimiciii.prescriptions')
rows = cur.fetchall()

In [47]:
prescriptionTable = pd.DataFrame(rows,columns=col_name)

In [46]:
col_name = []
for i in range(len(cur.description)):
    col_name.append(cur.description[i][0])
print (col_name)

['row_id', 'subject_id', 'hadm_id', 'icustay_id', 'startdate', 'enddate', 'drug_type', 'drug', 'drug_name_poe', 'drug_name_generic', 'formulary_drug_cd', 'gsn', 'ndc', 'prod_strength', 'dose_val_rx', 'dose_unit_rx', 'form_val_disp', 'form_unit_disp', 'route']


In [48]:
prescriptionTable.head(10)

,row_id,subject_id,hadm_id,icustay_id,startdate,enddate,drug_type,drug,drug_name_poe,drug_name_generic,formulary_drug_cd,gsn,ndc,prod_strength,dose_val_rx,dose_unit_rx,form_val_disp,form_unit_disp,route
0,2370392,24915,194424,267059.0,2182-12-04,2183-01-15,MAIN,Senna,Senna,Senna,SENN5L,002922,00034180080,5ml Oral Syringe,1,TAB,1,SYR,NG
1,2370389,24915,194424,267059.0,2182-12-04,2183-01-15,MAIN,Docusate Sodium (Liquid),Docusate Sodium (Liquid),Docusate Sodium (Liquid),DOCU100L,003017,51079033530,100MG UD CUP,100,mg,1,UDCUP,PO
2,2371058,24915,194424,267059.0,2182-12-05,2182-12-06,MAIN,Magnesium Citrate,Magnesium Citrate,Magnesium Citrate,MAGCITL,003025,00436515110,300mL Bottle,300,ml,1,BTL,PO
3,2371057,24915,194424,267059.0,2182-12-05,2182-12-06,MAIN,Mineral Oil,Mineral Oil,Mineral Oil,MINE30L,002997,00121046230,30mL Cup,15-30,ml,0.5-1,UDCUP,PO
4,2371056,24915,194424,267059.0,2182-12-05,2182-12-17,MAIN,Lactulose,Lactulose,Lactulose,LACT30L,003067,00054848616,20GM/30ML CUP,60,ml,2,UDCUP,PO
5,2368327,24915,194424,267059.0,2182-12-07,2182-12-08,BASE,D5W,None,None,D5W1000,001972,00338001704,1000mL Bag,1000,ml,1000,ml,IV
6,2366830,24915,194424,267059.0,2182-12-07,2182-12-08,MAIN,Potassium Chloride,None,None,KCL20PM,045309,00338070341,20mEq/50mL Premix,20,mEq,1,BAG,IV
7,2368325,24915,194424,267059.0,2182-12-07,2182-12-08,BASE,SW,None,None,KCLBASE2,None,0,50ml Bag,50,ml,50,ml,IV
8,2366832,24915,194424,267059.0,2182-12-07,2182-12-08,MAIN,Sodium Bicarbonate,None,None,NABC50I,001185,00074662502,50mEq Vial,150,mEq,3,VIAL,IV
9,2371059,24915,194424,267059.0,2182-12-07,2182-12-09,MAIN,Phytonadione,Phytonadione,Phytonadione,PHYT5,002305,00006004368,5mg Tab PPK,10,mg,2,TAB,NG


## Convert NDC to Rxcui and add it in to prescription table

In [54]:
ndc_lst = prescriptionTable['ndc'].unique()
url_lst = ['http://rxnav.nlm.nih.gov/REST/rxcui.json?idtype=NDC&id={0}'.format(ndc)
           for ndc in ndc_lst]

loop = asyncio.get_event_loop()
future = asyncio.ensure_future(run(url_lst))
responses = loop.run_until_complete(future)

total time: 30.88156501300182


In [83]:
def get_rxnormId_by_ndc(response):
    try:
        #class_lst = [x['rxclassMinConceptItem']['classId'] for x in json.loads(response)['rxclassDrugInfoList']['rxclassDrugInfo']]
        return json.loads(response)['idGroup']['rxnormId'][0]
    except:
        return ''

In [84]:
Rx_code_lst = [get_rxnormId_by_ndc(resp) for resp in responses]

In [86]:
ndc_rc_code_lst_map = {ndc: rx
                          for ndc, rx in zip(ndc_lst, Rx_code_lst)}

In [90]:
NDC_RX = pd.DataFrame({'NDC': ndc_lst,
              'Rx_code': Rx_code_lst})

In [95]:
prescriptionTable0 = prescriptionTable

In [96]:
prescriptionTable = prescriptionTable.assign(rxcui = prescriptionTable['ndc'].map(ndc_rc_code_lst_map))

In [97]:
prescriptionTable.head(20)

,row_id,subject_id,hadm_id,icustay_id,startdate,enddate,drug_type,drug,drug_name_poe,drug_name_generic,formulary_drug_cd,gsn,ndc,prod_strength,dose_val_rx,dose_unit_rx,form_val_disp,form_unit_disp,route,rxcui
0,2370392,24915,194424,267059.0,2182-12-04,2183-01-15,MAIN,Senna,Senna,Senna,SENN5L,002922,00034180080,5ml Oral Syringe,1,TAB,1,SYR,NG,
1,2370389,24915,194424,267059.0,2182-12-04,2183-01-15,MAIN,Docusate Sodium (Liquid),Docusate Sodium (Liquid),Docusate Sodium (Liquid),DOCU100L,003017,51079033530,100MG UD CUP,100,mg,1,UDCUP,PO,
2,2371058,24915,194424,267059.0,2182-12-05,2182-12-06,MAIN,Magnesium Citrate,Magnesium Citrate,Magnesium Citrate,MAGCITL,003025,00436515110,300mL Bottle,300,ml,1,BTL,PO,
3,2371057,24915,194424,267059.0,2182-12-05,2182-12-06,MAIN,Mineral Oil,Mineral Oil,Mineral Oil,MINE30L,002997,00121046230,30mL Cup,15-30,ml,0.5-1,UDCUP,PO,
4,2371056,24915,194424,267059.0,2182-12-05,2182-12-17,MAIN,Lactulose,Lactulose,Lactulose,LACT30L,003067,00054848616,20GM/30ML CUP,60,ml,2,UDCUP,PO,
5,2368327,24915,194424,267059.0,2182-12-07,2182-12-08,BASE,D5W,None,None,D5W1000,001972,00338001704,1000mL Bag,1000,ml,1000,ml,IV,1795612
6,2366830,24915,194424,267059.0,2182-12-07,2182-12-08,MAIN,Potassium Chloride,None,None,KCL20PM,045309,00338070341,20mEq/50mL Premix,20,mEq,1,BAG,IV,1860466
7,2368325,24915,194424,267059.0,2182-12-07,2182-12-08,BASE,SW,None,None,KCLBASE2,None,0,50ml Bag,50,ml,50,ml,IV,
8,2366832,24915,194424,267059.0,2182-12-07,2182-12-08,MAIN,Sodium Bicarbonate,None,None,NABC50I,001185,00074662502,50mEq Vial,150,mEq,3,VIAL,IV,
9,2371059,24915,194424,267059.0,2182-12-07,2182-12-09,MAIN,Phytonadione,Phytonadione,Phytonadione,PHYT5,002305,00006004368,5mg Tab PPK,10,mg,2,TAB,NG,


## Convert Rxcui to ADC and check whether the drug is Anibiotics, and add it into table

In [100]:
rxcui_lst = prescriptionTable['rxcui'].unique()
url_lst = ['https://rxnav.nlm.nih.gov/REST/rxclass/class/byRxcui.json?rxcui={}&relaSource=ATC'.format(rxcui)
           for rxcui in rxcui_lst]

loop = asyncio.get_event_loop()
future = asyncio.ensure_future(run(url_lst))
responses = loop.run_until_complete(future)

total time: 66.3630501080006


In [101]:
def get_ATC_code_lst(response):
    try:
        class_lst = [x['rxclassMinConceptItem']['classId'] for x in json.loads(response)['rxclassDrugInfoList']['rxclassDrugInfo']]
        return class_lst
    except:
        return []

ATC_code_lst_lst = [get_ATC_code_lst(resp) for resp in responses]

In [102]:
pd.DataFrame({'rxcui': rxcui_lst,
                   'ATC_code_lst': ATC_code_lst_lst})

,ATC_code_lst,rxcui
0,[],
1,"[B05CX, V04CA, V06DC]",1795612
2,"[A12BA, B05XA]",1860466
3,"[A12CA, B05CB, B05XA]",1807631
4,[J01XX],404652
5,"[A06AD, A12CC, B05XA, D11AX, V04CC]",1658221
6,"[B05CX, V04CA, V06DC]",1795610
7,"[A12BA, B05XA]",312504
8,[V03AE],209011
9,[C02DB],966571


In [103]:
ab_lst = [any(map(lambda x: x.startswith('J01'), ATC_code_lst)) 
           for ATC_code_lst in ATC_code_lst_lst]

In [104]:
pd.DataFrame({'rxcui': rxcui_lst,
                   'ATC_code_lst': ATC_code_lst_lst,
                   'ab': ab_lst})

,ATC_code_lst,ab,rxcui
0,[],False,
1,"[B05CX, V04CA, V06DC]",False,1795612
2,"[A12BA, B05XA]",False,1860466
3,"[A12CA, B05CB, B05XA]",False,1807631
4,[J01XX],True,404652
5,"[A06AD, A12CC, B05XA, D11AX, V04CC]",False,1658221
6,"[B05CX, V04CA, V06DC]",False,1795610
7,"[A12BA, B05XA]",False,312504
8,[V03AE],False,209011
9,[C02DB],False,966571


In [105]:
ab_map = {rxcui: ab for rxcui, ab in zip(rxcui_lst, ab_lst)}

In [112]:
prescriptionTable = prescriptionTable.assign(Antibiotics = prescriptionTable['rxcui'].map(ab_map))

## The table with anibiotics
## It seems sth different with PostDoc's result. His prescription table has 3500 unique Rxcui code, while mine is only 1800 or so.
## Garrett, can you help me check the first 500 rows manually. Coz, I can't tell whether a drug is AB or not from its name

In [113]:
prescriptionTable.head(500)

,row_id,subject_id,hadm_id,icustay_id,startdate,enddate,drug_type,drug,drug_name_poe,drug_name_generic,...,gsn,ndc,prod_strength,dose_val_rx,dose_unit_rx,form_val_disp,form_unit_disp,route,rxcui,Antibiotics
0,2370392,24915,194424,267059.0,2182-12-04,2183-01-15,MAIN,Senna,Senna,Senna,...,002922,00034180080,5ml Oral Syringe,1,TAB,1,SYR,NG,,False
1,2370389,24915,194424,267059.0,2182-12-04,2183-01-15,MAIN,Docusate Sodium (Liquid),Docusate Sodium (Liquid),Docusate Sodium (Liquid),...,003017,51079033530,100MG UD CUP,100,mg,1,UDCUP,PO,,False
2,2371058,24915,194424,267059.0,2182-12-05,2182-12-06,MAIN,Magnesium Citrate,Magnesium Citrate,Magnesium Citrate,...,003025,00436515110,300mL Bottle,300,ml,1,BTL,PO,,False
3,2371057,24915,194424,267059.0,2182-12-05,2182-12-06,MAIN,Mineral Oil,Mineral Oil,Mineral Oil,...,002997,00121046230,30mL Cup,15-30,ml,0.5-1,UDCUP,PO,,False
4,2371056,24915,194424,267059.0,2182-12-05,2182-12-17,MAIN,Lactulose,Lactulose,Lactulose,...,003067,00054848616,20GM/30ML CUP,60,ml,2,UDCUP,PO,,False
5,2368327,24915,194424,267059.0,2182-12-07,2182-12-08,BASE,D5W,None,None,...,001972,00338001704,1000mL Bag,1000,ml,1000,ml,IV,1795612,False
6,2366830,24915,194424,267059.0,2182-12-07,2182-12-08,MAIN,Potassium Chloride,None,None,...,045309,00338070341,20mEq/50mL Premix,20,mEq,1,BAG,IV,1860466,False
7,2368325,24915,194424,267059.0,2182-12-07,2182-12-08,BASE,SW,None,None,...,None,0,50ml Bag,50,ml,50,ml,IV,,False
8,2366832,24915,194424,267059.0,2182-12-07,2182-12-08,MAIN,Sodium Bicarbonate,None,None,...,001185,00074662502,50mEq Vial,150,mEq,3,VIAL,IV,,False
9,2371059,24915,194424,267059.0,2182-12-07,2182-12-09,MAIN,Phytonadione,Phytonadione,Phytonadione,...,002305,00006004368,5mg Tab PPK,10,mg,2,TAB,NG,,False


In [111]:
prescriptionTable['Antibiotics'] = 'True'

In [117]:
newTable = prescriptionTable.loc[prescriptionTable['Antibiotics'] == True, 'drug']

In [128]:
pd.DataFrame(pd.DataFrame(newTable)['drug'].unique()).to_csv('ab.csv')

In [129]:
prescriptionTable.to_csv('newPrescription.csv')